# Final Project - Criteo Labs Display Advertising Challenge
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2019`__

__`Team: Chi Iong Ansjory, Catherine Cao, Scott Xu`__

Table of Content:
- [0. Background](#background)
- [1. Question Formulation](#question_formulation)
- [2. Algorithm Explanation](#algorithm_explanation)
- [3. EDA & Discussion of Challenges](#eda_challenges)
- [4. Algorithm Implementation](#algorithm_implementation)
- [5. Application of Course Concepts](#course_concepts_application)

<a id='background'></a>
# 0. Background

Criteo Labs is a leading global technology company that specializes in performance display advertising, working with over 4,000 e-commerce companies around the world. Their technology takes an algorithmic approach to determining what user they show an advertisement to, when, and for what products. For billions of unique advertisements that are created and displayed at lightning fast speeds every day.

Display advertising is a billion dollar effort and one of the central uses of Machine Learning on the Internet. However, its data and methods are usually kept confidential. Through the Kaggle research competition, Criteo Labs is sharing a week’s worth of data for participants to develop models predicting advertisement click-through rate (CTR). Given a user and the page being visited, what is the probability that the user will click on a given advertisement?

Source: https://www.kaggle.com/c/criteo-display-ad-challenge

For the dataset, the smaller version is no longer available from Kaggle. The full-size version needs to be used instead from Criteo Labs.

Source: https://www.kaggle.com/c/criteo-display-ad-challenge/data (smaller version - obsoleted); http://labs.criteo.com/2014/02/kaggle-display-advertising-challenge-dataset/ (full-size version)

### Notebook Set-Up

In [1]:
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

import os

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "fpj_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

from pyspark.sql.types import *
import pyspark.sql.functions as F

<a id='question_formulation'></a>
# 1. Question Formulation

The goal of this analysis is to benchmark the most accurate ML algorithms for CTR estimation.

Understanding the drivers of what influences an individual user to click on a link or not has billions of dollars worth of implications. If we are able to develop a model that predicts user click through rates, the digital advertising market could become more efficient as billions of dollars is wasted annually on mistargeted users. Search engine marketers and analysts frequently perform this type of analysis to reduce one of the most expensive elements of eCommerce, which is called the Cost of Acquisition, or CAC. Particularly for SaaS and digital goods and services, CAC can constitute the majority of costs of goods sold. By being able to target interested customers better, thereby reducing the CAC of a product, less ad-spend goes wasted and the profit margins on each unit sold increases. Consequently businesses have a strong interest in ensuring that ads are properly targeted and CTRs are high.

In an ideal world with perfect information, we would be able to serve up advertisements that have a 100% CTR; that is, each ad we serve results in a click. This would then enable us to focus on conversion tactics that follow the initial ad click. According to WordStream, an online advertising consultancy, a CTR of over 2% for a search advertisement would be terrific. Typically, CTRs are divided up by industry, as each industry's customer has a different profile. For Display Advertisements, which is what Criteo primarily sells, a CTR of above 0.4% would be very good, again, depending on industry.

Source: https://www.wordstream.com/average-ctr

<a id='algorithm_explanation'></a>
# 2. Algorithm Explanation

Based on the volume of the full-size dataset (11GB), we decided to leverage PySpark and Spark ML Library (MLLib) packages. In addressing our project question of correctly predicting CTR, we evaluated following 3 algorithms: Logistic Regression (LR), Gradient Boosted Tree (GBT), and Random Forest (RF).

Source: https://spark.apache.org/docs/2.3.0/ml-guide.html

Logistic Regression is a popular method to predict a categorical response. It is a special case of Generalized Linear models that predicts the probability of the outcomes. If we model this sample data with a logistic regression, we assume that there are parameters $\beta_1$ to $\beta_{22}$ that:

$$ y(clicked) = \frac{exp(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_{22} x_{22})}{1 + exp(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_{22} x_{22})} $$

This is in the matrix form of:

$$ \theta^T = \big[ \beta_0 \beta_1 ... \beta_{22} \big] $$

Let sigmoid function $G(z)$ be:

$$ G(z) = \frac{1}{1 + e^{-z}} $$

Each prediction will be:

$$ y_i = G(x_i \theta^T) $$

The log loss function for the logistic regression is defined as:

$$ J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} (y_i log(H(x_i)) - (1-y_i)log(1-H(x_i)))$$

We can think of the log loss function as the error of our predictions and the process of solving for $\beta$'s as to minimize the log loss function. In order to do that, we can use the method of gradient descent where we iteratively adjust the coefficients $\theta$ to the direction that decreases the log loss function. The direction is given to us by the derivative of the log loss function.

Taking a derivative of the log loss function with respect to $\theta$ to get the gradient:

$$ \frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} (H(x_i)-y_i) * x_{ij}$$

With the gradient computed in every iteration, we get the next $\theta$ by:

$$ \theta_{new} = \theta - \alpha * \frac{\partial J(\theta)}{\partial \theta_j} $$

where $\alpha$ is the learning rate for the gradient descent

<a id='eda_challenges'></a>
# 3. EDA & Discussion of Challenges

The main challenges are the dataset given for this analysis has no column labels. We can't leverage any of our pre-existing knowledge about how online ads are served and CTR is computed in understanding the data. This means we have to put in extra effort in analyzing the data so we can understand the relationships between different features in the dataset and process them appropriately.

__train.txt__ is a dataset of 11GB and __test.txt__ is a dataset of 1.5GB. There is an additional column for label in __train.txt__ but not in __test.txt__. So, __train.txt__ will be splitted 70%/30% for training data and test data.

First of all, loading the __train.txt__ raw dataset into RDD, and look at the first 5 rows of data.

In [5]:
# load the raw data into an RDD
trainRDD = sc.textFile('data/train.txt')

In [6]:
# take a look at raw data
trainRDD.take(5)

['0\t1\t1\t5\t0\t1382\t4\t15\t2\t181\t1\t2\t\t2\t68fd1e64\t80e26c9b\tfb936136\t7b4723c4\t25c83c98\t7e0ccccf\tde7995b8\t1f89b562\ta73ee510\ta8cd5504\tb2cb9c98\t37c9c164\t2824a5f6\t1adce6ef\t8ba8b39a\t891b62e7\te5ba7672\tf54016b9\t21ddcdc9\tb1252a9d\t07b5194c\t\t3a171ecb\tc5c50484\te8b83407\t9727dd16',
 '0\t2\t0\t44\t1\t102\t8\t2\t2\t4\t1\t1\t\t4\t68fd1e64\tf0cf0024\t6f67f7e5\t41274cd7\t25c83c98\tfe6b92e5\t922afcc0\t0b153874\ta73ee510\t2b53e5fb\t4f1b46f3\t623049e6\td7020589\tb28479f6\te6c5b5cd\tc92f3b61\t07c540c4\tb04e4670\t21ddcdc9\t5840adea\t60f6221e\t\t3a171ecb\t43f13e8b\te8b83407\t731c3655',
 '0\t2\t0\t1\t14\t767\t89\t4\t2\t245\t1\t3\t3\t45\t287e684f\t0a519c5c\t02cf9876\tc18be181\t25c83c98\t7e0ccccf\tc78204a1\t0b153874\ta73ee510\t3b08e48b\t5f5e6091\t8fe001f4\taa655a2f\t07d13a8f\t6dc710ed\t36103458\t8efede7f\t3412118d\t\t\te587c466\tad3062eb\t3a171ecb\t3b183c5c\t\t',
 '0\t\t893\t\t\t4392\t\t0\t0\t0\t\t0\t\t\t68fd1e64\t2c16a946\ta9a87e68\t2e17d6f6\t25c83c98\tfe6b92e5\t2e8a689b\t0b15387

__train.txt__ is essentially a text file with tab-separated values, in which all tab-delimited dataset need to be splitted into variables. A header also needs to be added for the identified target numeric variable, 13 numeric features, and 26 categorical features. Then, create a train dataframe with the header, cast integer variables to its type, verify the schema, and the total count of data.

In [7]:
# split variables
temp_var = trainRDD.map(lambda x: x.split("\t"))

In [8]:
# create header for dataset with numeric and categorical features
numeric_features = ['I'+str(i) for i in range(1,14)]
categorical_features = ['C'+str(i) for i in range(1,27)]
header = ['target'] + numeric_features + categorical_features

In [9]:
# create and look at pyspark dataframe
train_df = temp_var.toDF(header)
train_df.show()

+------+---+---+---+---+-----+---+---+---+----+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|target| I1| I2| I3| I4|   I5| I6| I7| I8|  I9|I10|I11|I12|I13|      C1|      C2|      C3|      C4|      C5|      C6|      C7|      C8|      C9|     C10|     C11|     C12|     C13|     C14|     C15|     C16|     C17|     C18|     C19|     C20|     C21|     C22|     C23|     C24|     C25|     C26|
+------+---+---+---+---+-----+---+---+---+----+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|     0|  1|  1|  5|  0| 1382|  4| 15|  2| 181|  1|  2|   |  2|68fd1e64|80e26c9b|fb936136|7b4723c4|25c83c9

In [10]:
# cast integer variables to integer type
for var in ['target'] + numeric_features:
    train_df = train_df.withColumn(var, train_df[var].cast(IntegerType()))

In [11]:
# check schema
train_df.printSchema()

root
 |-- target: integer (nullable = true)
 |-- I1: integer (nullable = true)
 |-- I2: integer (nullable = true)
 |-- I3: integer (nullable = true)
 |-- I4: integer (nullable = true)
 |-- I5: integer (nullable = true)
 |-- I6: integer (nullable = true)
 |-- I7: integer (nullable = true)
 |-- I8: integer (nullable = true)
 |-- I9: integer (nullable = true)
 |-- I10: integer (nullable = true)
 |-- I11: integer (nullable = true)
 |-- I12: integer (nullable = true)
 |-- I13: integer (nullable = true)
 |-- C1: string (nullable = true)
 |-- C2: string (nullable = true)
 |-- C3: string (nullable = true)
 |-- C4: string (nullable = true)
 |-- C5: string (nullable = true)
 |-- C6: string (nullable = true)
 |-- C7: string (nullable = true)
 |-- C8: string (nullable = true)
 |-- C9: string (nullable = true)
 |-- C10: string (nullable = true)
 |-- C11: string (nullable = true)
 |-- C12: string (nullable = true)
 |-- C13: string (nullable = true)
 |-- C14: string (nullable = true)
 |-- C15: string

In [12]:
# how many records do we have
total_count = train_df.count()
total_count

45840617

Now, the next challenges are to decide how to handle missing values in the variables. There are missing values for both numeric and categorical features, but not the target variable. For missing numeric variables, they show up as __NaN__. For some reason, missing categorical variables are not recognized as __NaN__, replace them with __None__. The distribution of target variable is fairly balanced. 

In [19]:
# take a look at data
pd.DataFrame(train_df.take(5), columns=train_df.columns).transpose()

,0,1,2,3,4
target,0,0,0,0,0
I1,1,2,2,NaN,3
I2,1,0,0,893,-1
I3,5,44,1,NaN,NaN
I4,0,1,14,NaN,0
I5,1382,102,767,4392,2
I6,4,8,89,NaN,0
I7,15,2,4,0,3
I8,2,2,2,0,0
I9,181,4,245,0,0


In [20]:
# look at missing statistics
non_missing = train_df.summary("count")
missing_summary = non_missing.toPandas().drop(['summary'], axis=1).transpose()
missing_summary.columns = ['non-missing']
missing_summary['missing'] = total_count - missing_summary['non-missing'].astype('int64')
missing_summary['missing_pct'] = missing_summary['missing'] / total_count
missing_summary.sort_values(['missing_pct'], ascending=False)

,non-missing,missing,missing_pct
I12,10768965,35071652,0.765078
C22,10885544,34955073,0.762535
I1,25047061,20793556,0.453606
I10,25047061,20793556,0.453606
C26,25667759,20172858,0.440065
C25,25667759,20172858,0.440065
C20,25667759,20172858,0.440065
C19,25667759,20172858,0.440065
I6,35588289,10252328,0.223652
I4,35903248,9937369,0.216781


In [22]:
# replace missing categorical variables not recognized as NA with the None
train_df = train_df.replace('', None, categorical_features)

In [ ]:
# look at target variable distribution - fairly balanced
train_df.groupBy('target').count().toPandas()

In [25]:
# take 1% of data as a sample for more EDA
sample = train_df.sample(0.01, 666)
sample.count()

459205

<a id='algorithm_implementation'></a>
# 4. Algorithm Implementation

<a id='course_concepts_application'></a>
# 5. Application of Course Concepts